In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import cv2
import numpy as np

import io
from IPython.display import display
import fileupload

%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib

from IPython.display import clear_output
    
src_img = None

def find_face(original_img, img, scale_factor, min_neighbors):
    face_cascade = cv2.CascadeClassifier('cascades/haarcascade_frontalface_alt.xml')
    faces = face_cascade.detectMultiScale(original_img, scale_factor, min_neighbors)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,215,0),5)
        roi_color = img[y:y+h, x:x+w]

    return img
    
    
def find_eyes(original_img, img, scale_factor, min_neighbors):
    eyes_cascade = cv2.CascadeClassifier('cascades/haarcascade_eye.xml')
    eyes = eyes_cascade.detectMultiScale(original_img, scale_factor, min_neighbors)

    for (x,y,w,h) in eyes:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),5)
        roi_color = img[y:y+h, x:x+w]
        
    return img
    
def read_img(path):
    img = cv2.imread(path)
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img
    
def show_img(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    plt.figure(figsize=(20,10))
    plt.imshow(img)

def process_img(face_scale_factor, face_neighbors, eyes_scale_factor, eyes_neighbors):
    if src_img is not None:
        img = src_img.copy()
        original_img = src_img.copy()
        img = find_eyes(original_img, img, eyes_scale_factor, eyes_neighbors)
        img = find_face(original_img, img, face_scale_factor, face_neighbors)
        show_img(img)
    
def reload_img(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    plt.figure(figsize=(20,10))
    plt.draw

        
def sliders():
    
    style = {'description_width': 'initial'}
    interact(process_img, 
             face_scale_factor=widgets.FloatSlider(
                    value=1.05,
                    min=1.01,
                    max=1.5,
                    step=0.01,
                    description='Face Scale Factor:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='.2f',
                    style=style
                    ),
             face_neighbors=widgets.IntSlider(
                    value=4,
                    min=2,
                    max=9,
                    step=1,
                    description='Face Min Neighbors:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d',
                    style=style
                    ),
             eyes_scale_factor=widgets.FloatSlider(
                    value=1.3,
                    min=1.01,
                    max=1.5,
                    step=0.01,
                    description='Eyes Scale Factor:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='.2f',
                    style=style
                    ),
             eyes_neighbors=widgets.IntSlider(
                    value=4,
                    min=2,
                    max=9,
                    step=1,
                    description='Eyes Min Neighbors:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d',
                    style=style
                    )
            );
    


def _upload():
#     widgets.Label(value='Wybierz z dysku zdjęcie, na którym chcesz pracować: ')
    _upload_widget = fileupload.FileUploadWidget()
    
    def data_uri_to_cv2_img(uri):
        encoded_data = uri.split(',')[1]
        nparr = np.fromstring(encoded_data.decode('base64'), np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        return img
    

    def _cb(change):
        filename = change['owner'].filename
        global src_img
        src_img = data_uri_to_cv2_img(change['owner'].data_base64)
        clear_output()
        _upload()
        sliders()      
        

    _upload_widget.observe(_cb, names='data')
    display(_upload_widget)
     
_upload()  
sliders()




FileUploadWidget(label=u'Browse', _dom_classes=('widget_item', 'btn-group'))